# In-class work 1: Introduction to training neural networks

Okay, now that we've covered some data science basics, let's get a very baseline neural network training pipeline going. 

**Important:** Make sure this notebook is pointing toward your custom `conda` kernel (top right drop-down menu).

<a name='section_0'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">0. Installing pytorch</h2>

Installation recipes can change over time for popular ML repositories, so instead of giving you a formula to memorize here, I'm going to recommend you go straight to the source. Find the Pytorch documentation and look at the installation instructions. Make sure you can run the cell below before proceeding.

**NOTE:** do not use `pip3 install` directly in a notebook cell -- refer to the previous notebook for guidance on how to install packages directly from a notebook cell, or use a Terminal window instead.

In [ ]:
%pip install torch torchvision

In [ ]:
### test that torch will work 

import torch

x = torch.rand(5, 3)
print(x)

In [ ]:
### here's the full cell of imports 

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

### set plot resolution
%config InlineBackend.figure_format = 'retina'

### set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_1'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">1. Generating a dataset</h2>

First, we'll generate one of the simplest toy datasets there is: two Gaussian distributions. Let's say that they describe our signal and our background events, but that there are only 2 parameters we can measure ($\theta_1$ and $\theta_2$).

In [ ]:
### Two 2D Gaussian distributions with the same number of events
n_events = 2000

### Background is sampled from N(mu0, Sigma0)

mu0 = np.array([-1.0, -1.0])
cov0 = np.array([[1.0, 0.2],
                 [0.2, 1.2]])
x0 = np.random.multivariate_normal(mu0, cov0, size=n_events)
y0 = np.zeros((n_events, 1), dtype=np.float32)

### Signal is sampled from N(mu1, Sigma1)
mu1 = np.array([1.0, 1.0])
cov1 = np.array([[1.3, -0.3],
                 [-0.3, 0.8]])
x1 = np.random.multivariate_normal(mu1, cov1, size=n_events)
y1 = np.ones((n_events, 1), dtype=np.float32)

X = np.vstack([x0, x1]).astype(np.float32)
y = np.vstack([y0, y1]).astype(np.float32)

### Put these together and shuffle
idx = np.random.permutation(len(X))
X = X[idx] # inputs
y = y[idx] # labels

print("Data shapes:", X.shape, y.shape)

Let's put it into a Pandas dataframe for legibility:

In [ ]:
df = pd.DataFrame(np.hstack([X,y]), columns = ["theta_1", "theta_2", "label"])

In [ ]:
df

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(df[df.label != True].theta_1, df[df.label != True].theta_2, s=6, color="grey", alpha=0.5, label="Background")
plt.scatter(df[df.label == True].theta_1, df[df.label == True].theta_2, s=6, color="crimson", alpha=0.5, label="Signal")
plt.legend()
plt.title("Visualizing our datasets")
plt.xlabel(r"$\theta_1$"); plt.ylabel(r"$\theta_2$")
plt.show()

<a name='section_2'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">2. Defining a neural network model</h2>

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32):
        super().__init__()
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # output is 1-dimensional for binary classification
        )
    def forward(self, x):
        return self.MLP(x)

<a name='section_3'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">3. Preprocessing</h2>

Key steps include splitting into train/validation/test sets, standardizing data, and building dataloaders.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["label"])  # only look at the features, not the labels
y = df["label"]                 # labels

### Take the first 70% for training
X_train, X_valtest, y_train, y_valtest = train_test_split(
    X, y, test_size=0.3)

### Then evenly split the remaining 30% into validation & test set 
X_val, X_test, y_val, y_test = train_test_split(
    X_valtest, y_valtest, test_size=0.5
)

print(f"Train set has {len(X_train)} events.")
print(f"Validation set has {len(X_val)} events.")
print(f"Test set has {len(X_test)} events.")

You can standardize your data using a "z-score" manually by subtracting the mean and dividing by the standard deviation. 

Alternatively, you can use the StandardScaler plugin to do this automatically:

In [ ]:
from sklearn.preprocessing import StandardScaler

### initialize the scaler
scaler = StandardScaler()

### IMPORTANT -- you only want to define your scaling ("fit_transform") based on your training dataset.
X_train_scaled = scaler.fit_transform(X_train)

### Now re-use the same transformation on validation & test sets ("transform")
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Train mean (before scaling):", np.array(X_train.mean(axis=0)))
print("Train mean (after scaling) -- each dimension should be close to 0:", X_train_scaled.mean(axis=0))
print("Train std (before scaling)", np.array(X_train.std(axis=0)))
print("Train std (after scaling) -- each dimension should be close to 1:", X_train_scaled.std(axis=0))

In [ ]:
### batch size is often an important hyperparameter. we'll use 32 as our default for now. 

batch_size = 32

train_ds = TensorDataset(torch.from_numpy(X_train_scaled), torch.from_numpy(y_train.values.astype("float32")).view(-1, 1))
val_ds   = TensorDataset(torch.from_numpy(X_val_scaled),   torch.from_numpy(y_val.values.astype("float32")).view(-1, 1))
test_ds  = TensorDataset(torch.from_numpy(X_test_scaled),  torch.from_numpy(y_test.values.astype("float32")).view(-1, 1))

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

Test one of these loaders:

In [ ]:
for x_batch, y_batch in train_loader:
    print(x_batch.shape, y_batch.shape)
    break

<a name='section_4'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">4. Training</h2>

Let's play with tracking our model training using `livelossplot`.

In [ ]:
%pip install livelossplot

In [ ]:
from livelossplot import PlotLosses

In [ ]:
### Check for available GPUs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

### Initialize the model, optimizer, and loss function
model = MLP(input_dim=2, hidden_dim=32).to(device)
model = model.to(device) # move onto the GPU, if present
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

liveloss = PlotLosses(figsize=(9, 4)) 
logs = {}


### Training loop
for epoch in range(10):

    ### train
    model.train()
    total_train_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    
    train_loss_per_batch = total_train_loss / len(train_loader)
    logs['loss'] = train_loss_per_batch

    ### validate
    model.eval()
    total_val_loss = 0
    for x_batch, y_batch in val_loader:
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        total_val_loss += loss.item()
    
    val_loss_per_batch = total_val_loss / len(val_loader)
    logs['val_loss'] = val_loss_per_batch
    
    liveloss.update(logs)
    
    liveloss.send()
    print(f'Epoch {epoch}, Loss: {loss.item()}')

#### Using wandb

Skip this for now, but if you have time once you finish this notebook, try connecting the notebook to Wandb to have the option of tracking your runs: 
- https://docs.wandb.ai/guides/track/jupyter/ 

In [ ]:
# %pip install wandb -qqq

In [ ]:
# import wandb
# wandb.login()

In [ ]:
# wandb.init(
#     project="phys-805-test",
#     config={
#         "batch_size": 128,
#         "learning_rate": 0.01,
#         "dataset": "gaussians",
#     },
# )

<a name='section_5'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">5. Evaluating model performance</h2>

The loss alone might not be a very intuitive metric for understanding our model performance. Let's evaluate the model on our holdout test set and get some new metrics.

In [ ]:
### set the threshold for your NN score (e.g. 50%)
threshold = 0.5 

### test set 
model.eval()
test_loss, test_acc, count = 0.0, 0.0, 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        
        ### Move the batch onto the GPU, if present
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
    
        out = model(x_batch)
        loss = loss_fn(out, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_acc  += (torch.sigmoid(out) > threshold).float().eq(y_batch).float().mean().item() * x_batch.size(0)
        count += x_batch.size(0)
        
print(f"Average test loss: {test_loss/count:.4f} | Test accuracy: {test_acc/count:.3f}")

### Further exercises:
- What is a simple baseline you could define here to get context for how well your model is doing? Implement this and use it as a comparison.
- Try modifying the hidden size, number of MLP layers, to see the effect on performance
- Try increasing the dataset size
- Try different learning rates
- What's the effect of changing your batch size?
- Try adding dropout
- Try making the task harder by making the Gaussians overlap and/or shifting their positions. Where does your model start to break down?
- Can you track an "accuracy" metric alongside the loss in your live tracking plots?
- Can you visualize the decision boundary in parameter space?